In [1]:
import csv
import random
import numpy as np

# Import datasets, classifiers and performance metrics
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [109]:
# candidate info
candidate_count = 0
candidates = []

incomplete_counts = None

# ballot mete data
total_voter_count = 0
total_vote_count = 0
unique_order_count = 0

# ballot data
# map a unique order of ballots to the total count of ballots in this order
 # format: tuple(first choice,second,third choice...): count
ballots = []

ballots_size = []

with open("data/dublin-north.soi", newline="\n") as data_file:
    data = csv.reader(data_file)
    i = 0
    for row in data:
        if i == 0:
            candidate_count = int(row[0])
            incomplete_counts = [0]*(candidate_count+1)
        elif i <= candidate_count:
            candidates.append(row[1])
        elif i == candidate_count + 1:
            total_voter_count = int(row[0])
            total_vote_count = int(row[1])
            unique_order_count = int(row[2])
        else:
            ballot = tuple(int(j) for j in row[1:])
            ballots.append([ballot, int(row[0])])
            ballots_size.append(len(ballot))
            incomplete_counts[len(ballot)] += int(row[0])
        i += 1

In [107]:
print(incomplete_counts)

[0, 1688, 2796, 12589, 7861, 6163, 3713, 2184, 1327, 686, 676, 597, 3662]


In [ ]:
# print(candidate_count)
# for key in ballots.keys():
#     print(key, ballots[key])
#     print(len(key))
# for ballot in ballots:
#     print(ballot, ballots[ballot])
#     assert len(ballot[0]) != 0
# print(train_data[0][:-1])

In [3]:
# unroll ballots
def unroll_ballots(ballots):
    unrolled_ballots = []
    for b in range(len(ballots)):
        for i in range(ballots[b][1]):
            unrolled_ballots.append(list(ballots[b][0]))
    return unrolled_ballots

# unrolled_ballots = unroll_ballots(ballots)
# print(len(unrolled_ballots), " ballots")

In [4]:
# get train and set labels
def known_target_split(unrolled_ballots, target_length):
    known_selections = []
    target_selection = []
    for b in range(len(unrolled_ballots)):
        if len(unrolled_ballots[b]) >= target_length:
            known_selections.append(list(unrolled_ballots[b][:target_length - 1]))
            target_selection.append(unrolled_ballots[b][target_length - 1])
    return known_selections, target_selection

# known_selections, target_selection = known_target_split(unrolled_ballots, 5)
# print(len(known_selections))

In [100]:
# divide into train and test set
known_selections, target_selection = known_target_split(unrolled_ballots, 2)
X_train, X_test, y_train, y_test = train_test_split(known_selections,target_selection, test_size = 0.2, random_state=1)

In [101]:
# Create a classifier: a support vector classifier
# clf = svm.SVC(gamma=0.15)
clf = LogisticRegression(penalty='l1', solver='liblinear', max_iter=800)

In [102]:
clf.fit(X_train, y_train)

LogisticRegression(max_iter=800, penalty='l1', solver='liblinear')

In [103]:
predicted = clf.predict(X_test)

In [104]:
print(f"{metrics.classification_report(y_test, predicted)}\n")

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       526
           2       0.00      0.00      0.00       872
           3       0.00      0.00      0.00       257
           4       0.00      0.00      0.00      1017
           5       0.00      0.00      0.00       217
           6       0.17      0.52      0.25      1298
           7       0.00      0.00      0.00       722
           8       0.00      0.00      0.00        79
           9       0.00      0.00      0.00      1129
          10       0.17      0.58      0.26      1397
          11       0.00      0.00      0.00       386
          12       0.00      0.00      0.00       889

    accuracy                           0.17      8789
   macro avg       0.03      0.09      0.04      8789
weighted avg       0.05      0.17      0.08      8789




/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [110]:
def train_svm(known_selections, target_selection):
    clf = svm.SVC(gamma=0.15)
#     X_train, X_test, y_train, y_test = train_test_split(known_selections,target_selection, test_size = 0.2, random_state=1)    
#     clf.fit(X_train, y_train)
    clf.fit(known_selections, target_selection)
    return clf
def train_lrg(known_selections, target_selection):
    clf = LogisticRegression(penalty='l1', solver='liblinear', max_iter=800)
    clf.fit(known_selections, target_selection)
    return clf

In [111]:
miss_count = 0
total_impute = 0

In [44]:
print(miss_count/total_impute)

5.733045695656384


In [112]:
# Return an order of ballot as a tuple


def predict(ballots, target_ballot, model):
    # finish this
    global miss_count
    global total_impute
    total_impute += 1
    
    target = model.predict_proba([target_ballot])
    
    target = target.tolist()[0]
#     print(target)
    cands = [(i+1,target[i]) for i in range(len(target))]
    cands.sort(key=lambda x: x[1])
    for c,v in cands:
        if c not in target_ballot:
            target = c
            break
        miss_count += 1
#     print(cands)
#     print(ballot)
#     print(target)
    improved_target_ballot = list(target_ballot)
    improved_target_ballot.append(target)
    
#     ##################### place holder ####################
#     temp = list(target_ballot)
#     temp.append(random.randint(1,12))
#     improved_target_ballot = tuple(temp)
#     #######################################################
    
    return improved_target_ballot

In [113]:
# this code will modify the data in "ballots". Need to rerun above data reading code to restore
# data in the dictionary.
for i in range(1, candidate_count):
    # unroll ballots
    unrolled_ballots = unroll_ballots(ballots)
#     cnt = 0
    # split into data and labels, where the selection at i + 1 place is the label, selections before
    # are the data
    known_selections, target_selection = known_target_split(unrolled_ballots, i + 1)
    print(len(known_selections))
    # train model for predicting the i+1 preference
#     model = train_svm(known_selections, target_selection)
    model = train_lrg(known_selections, target_selection)
    # update ballots with only i preference
#     keys = list(ballots.keys())
    for b in range(len(ballots)):
        ballot = ballots[b][0]
        if len(ballot) == i : #and ballots[b][0] != 0:
#             cnt += 1
            improved_ballot = predict(ballots, ballot, model)
            ballots[b][0] = improved_ballot
#             if i == 11 and improved_ballot[-1] == 10: print("inter")
#         elif i == 11 and ballot[-1] == 10: print("true")
#     if i == 11: print(cnt)
#             if improved_ballot in ballots:
#                 ballots[improved_ballot] += ballots[ballot]
#             else:
#                 ballots[improved_ballot] = ballots[ballot]
#             del ballots[ballot]
#             ballots[ballot] = 0 # set old count to 0, should then have no impact on final vote count
# print(ballots)

42254
39458
26869
19008
12845
9132
6948
5621
4935
4259
3662


In [122]:
# Return a dictionary mapping candidate number to score.
# Calculate candidate scores based on Borda's Rule.
def borda_rule_result(ballots,ballots_size=None,score_decay=1):
    candidate_scores = dict.fromkeys(range(1, candidate_count+1), 0)
    candidate_scores_imp = dict.fromkeys(range(1, candidate_count+1), 0)
#     print(candidate_scores)
    for b in range(len(ballots)):
        if b in drop_list: continue
        ballot = ballots[b][0]
        count = ballots[b][1]
        dec_cnt = 1
        for i in range(candidate_count):
            if not ballots_size is None and ballots_size[b] < i:
                candidate_scores[ballot[i]] += (candidate_count - 1 - i) * count * pow(score_decay, dec_cnt)
                candidate_scores_imp[ballot[i]] += (candidate_count - 1 - i) * count * pow(score_decay, dec_cnt)
                dec_cnt += 1
            else:
                candidate_scores[ballot[i]] += (candidate_count - 1 - i) * count
    return candidate_scores, candidate_scores_imp

In [125]:
for i in [0, 0.25, 0.5, 0.75, .85, .9, .95, 1]: 
    l,limp = borda_rule_result(ballots, ballots_size,i)
    print("ROUND")
    for i in range(1,candidate_count+1): print(l[i])
    
    
#     ln = sorted(l.items(), key=lambda item: item[1])
#     ln.reverse()
#     limpn = sorted(limp.items(), key=lambda item: item[1])
#     limpn.reverse()
#     print(limp)
#     print(l)
#     maxi = 0
#     can = 0
#     for c,v in l.items():
#         if v > maxi:
#             maxi = v
#             can = c
#     print(can)
        

ROUND
118091
190074
104947
217028
85746
202740
161865
53778
232282
265750
228098
198238
ROUND
120811.77555847168
192129.21697616577
122202.08659362793
220067.32676696777
86891.3108215332
207058.69563293457
164702.36960220337
75856.24024963379
236799.45217895508
270159.47554016113
230394.23303222656
203401.35278320312
ROUND
125156.5625
195742.126953125
143201.72265625
226398.59765625
90684.4453125
217604.48828125
173659.533203125
100870.30078125
247494.1171875
280948.58203125
233032.28125
214853.046875
ROUND
134192.0406036377
202416.405292511
172599.9055633545
241806.4511566162
103750.76907348633
240813.8755645752
197513.60386276245
132774.5549468994
269659.9928894043
301050.53120422363
236293.32360839844
234678.87329101562
ROUND
140952.60672690353
206562.34568711862
188865.39984346484
253179.41804354847
115477.65132915352
255955.75811737307
214479.80724087937
149410.11524507325
283553.1520519987
312810.90447665664
237921.46773814072
245663.47024719292
ROUND
145539.7526496775
209095.336

In [ ]:
def svt(ballots, ballots_size=[],score_decay=1):
    elim = [] # eliminated candidates
    while True:
        candidate_scores = dict.fromkeys(range(1, candidate_count+1), 0)
        for b in range(len(ballots)):
            if b in drop_list: continue
            ballot = ballots[b][0]
            count = ballots[b][1]
            for i in range(candidate_count):
                if ballot[i] not in elim:
                    if not ballots_size is None and ballots_size[b] < i:
                        candidate_scores[ballot[i]] += count * score_decay
                    else:
                        candidate_scores[ballot[i]] += count
                    break
        least = 0
        first = True
        for (i,v) in candidate_scores.items():
            if i not in elim:
                if first:
                    least = i
                    first = False
                elif candidate_scores[least] > v:
                    least = i
        # eliminate lowest
        print("Eliminate "+str(least))
        elim.append(least)
        # if 1 left, return as winner
        if len(elim) == candidate_count-1: return [x for x in range(1,candidate_count+1) if x not in elim][0]

In [ ]:
for i in [0, 0.25, 0.5, 0.75, 1]: print(svt(ballots, ballots_size, i))

In [ ]:
def irv(ballots, ballots_size=[],score_decay=1):
    elim = [] # eliminated candidates
    while True:
        candidate_scores = dict.fromkeys(range(1, candidate_count+1), 0)
        for b in range(len(ballots)):
            if b in drop_list: continue
            ballot = ballots[b][0]
            count = ballots[b][1]
            for i in range(candidate_count):
                if ballot[i] not in elim:
                    if not ballots_size is None and ballots_size[b] < i:
                        candidate_scores[ballot[i]] += count * score_decay
                    else:
                        candidate_scores[ballot[i]] += count
                    break
        least = 0
        best = 0
        first = True
        for (i,v) in candidate_scores.items():
            if i not in elim:
                if first:
                    least = i
                    best = i
                    first = False
                elif candidate_scores[least] > v:
                    least = i
                elif candidate_scores[best] < v:
                    best = i
        # if majority, return as winner
        if candidate_scores[best] > len(ballots)/2: return best
        # otherwise eliminate lowest and continue
        print("Eliminate "+str(least))
        elim.append(least)
        

In [ ]:
for i in [0, 0.25, 0.5, 0.75, 1]: print(irv(ballots, ballots_size, i))

In [120]:
def copeland(ballots, ballot_size=None, score_decay=1):
    
    candidate_scores = dict.fromkeys(range(1, candidate_count+1), dict.fromkeys(range(1, candidate_count+1), 0))
    # get dic where [i][j] -> # times i beats j
    for b in range(len(ballots)):
        if b in drop_list: continue
        ballot = ballots[b][0]
        count = ballots[b][1]
        for i in range(candidate_count):
            for j in range(i+1,candidate_count):
                # ballot[i] beats all ballot[j]
                if not ballots_size is None and ballots_size[b] < i:
                    candidate_scores[ballot[i]][ballot[j]] += count * score_decay
                else:
                    candidate_scores[ballot[i]][ballot[j]] += count
                
    
    
    candidate_pairwise_wins = dict.fromkeys(range(1, candidate_count+1), 0)
    
    # loop through all pairwise matchups
    for i in range(1, candidate_count+1):
        for j in range(i+1, candidate_count+1):
            if candidate_scores[i][j] > candidate_scores[j][i]:
                # i beats j
                candidate_pairwise_wins[i] += 1
            else: 
                candidate_pairwise_wins[j] += 1
                
    
    max = 1
    # get candidate with most pairwise wins
    print("ROUND")
    for i in range(1, candidate_count+1):
        if candidate_pairwise_wins[i] > candidate_pairwise_wins[max]:
            max = i
#         print(str(i) + " wins: " + str(candidate_pairwise_wins[i]))
        print(str(candidate_pairwise_wins[i]))
    return max
    
    
    

In [121]:
# Update decay with exopnential...
for i in [0, 0.25, 0.5, 0.75, .85, .9, .95, 1]: print(copeland(ballots, ballots_size, i))

ROUND
4
6
1
9
2
8
5
0
10
11
3
7
10
ROUND
2
5
3
9
1
8
6
0
10
11
4
7
10
ROUND
2
5
3
8
0
9
6
1
10
11
4
7
10
ROUND
1
4
3
8
0
9
6
2
10
11
5
7
10
ROUND
1
4
3
8
0
9
6
2
10
11
5
7
10
ROUND
1
3
4
8
0
9
6
2
10
11
5
7
10
ROUND
1
3
4
8
0
9
6
2
10
11
5
7
10
ROUND
1
3
4
8
0
9
6
2
10
11
5
7
10


In [ ]:
# from numpy import random
# import numpy as np
def reduce_ballots(ballots, ballots_size, alpha, beta):
    array = np.array(range(len(ballots)))
    random.shuffle(array)
    change_ballots = array.tolist()
    alpha_cnt = int(len(ballots) * alpha)
    for b in change_ballots[0:alpha_cnt]: # Change first alpha random ballots
        # Can we get original data set???
        # Assuming all ballots are complete...
        step = 1
        ballot = ballots[b][0]
        count = ballots[b][1]
        for i in range(candidate_count-1,0,-1):
            r = random.uniform(0,1)
            if r > beta^step:
                ballot = ballot[:-1]
                step += 1
            else:
                break
        ballots[b] = [ballot,count]
        ballots_size[b] = candidate_count - (step-1)
        
    return ballots, ballots_size
        

In [117]:
def drop_some(alpha):
    drop_list = []
    alpha_cnt = int(len(ballots) * alpha)
    print("Drop "+str(alpha_cnt)+" from "+str(total_voter_count))
    array = np.array(range(len(ballots)))
    random.shuffle(array)
    change_ballots = array.tolist()
    for b in change_ballots:
        if alpha_cnt <= 0: break
        if ballots_size[b] < candidate_count:
            drop_list.append(b)
            alpha_cnt -= ballots[b][1]
    return drop_list

In [119]:
# drop_list = drop_some(0.3)
drop_list = []

In [ ]:
for b,c in ballots:
    for i in range(len(b)):
        if b[i] in b[i+1:]: 
            print(b)